# Model experiments

This notebook is for experimenting with the choice of model and performance variations between different time periods and clinics.

1. Train and evaluate current best model on total dataset (without differentiating between clinics)
2. Apply IECV to different models and check heteregeneity with respect to clinics
3. Train models per clinic and check performance
4. Use clinic as categorical feature in tree based models and check performance

## Load packages

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import random

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import (
    StratifiedGroupKFold,
    TimeSeriesSplit,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler, SplineTransformer

from noshow.features.feature_pipeline import create_features
from noshow.model.predict import create_prediction
from noshow.preprocessing.load_data import (
    load_appointment_csv,
    process_appointments,
    process_postal_codes,
)


## Load data and split in to train and test

In [ ]:
featuretable = pd.read_parquet("../data/processed/featuretable.parquet")

featuretable["no_show"] = (
    featuretable["no_show"].replace({"no_show": "1", "show": "0"}).astype(int)
)
featuretable["hour"] = featuretable["hour"].astype("category")
featuretable["weekday"] = featuretable["weekday"].astype("category")

print(featuretable.dtypes)

X, y = featuretable.drop(columns="no_show"), featuretable["no_show"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, shuffle=False
)
train_groups = X.index.get_level_values("pseudo_id")

In [ ]:
lgboost_model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=300,
    categorical_features=["hour", "weekday"],
)

categorical_features = ["hour", "weekday"]
continuous_features = X.columns.difference(categorical_features)

preprocessor = ColumnTransformer(
    transformers=[
        (
            "continuous",
            Pipeline([("scaler", RobustScaler()), ("spline", SplineTransformer())]),
            continuous_features,
        ),
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

log_model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression(penalty=None)),
    ]
)

## Train and evaluate current best model on total dataset (without differentiating between clinics)

In [ ]:
def cv_auc_curve(X_train, y_train, model, cv, train_groups=None, title=None):
    fpr = {}
    tpr = {}
    roc_auc = {}
    test_indices = {}

    fig, ax = plt.subplots()

    for i, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train, train_groups)):
        X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_train_cv, y_train_cv)

        y_score = model.predict_proba(X_test_cv)[:, 1]
        fpr[i], tpr[i], _ = roc_curve(y_test_cv, y_score)
        roc_auc[i] = roc_auc_score(y_test_cv, y_score)
        test_indices[i] = test_idx

        ax.plot(fpr[i], tpr[i], c="b", alpha=0.15)

    ax.plot([0, 1], [0, 1], "k--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    # Add mean AUC and standard deviation to the legend
    mean_auc = np.mean(list(roc_auc.values()))
    std_auc = np.std(list(roc_auc.values()))
    ax.legend([f"ROC curve (AUC = {mean_auc:.3f} +/- {std_auc:.3f})"])
    if title:
        ax.set_title(title)
    fig.show()

    return roc_auc, test_indices

In [ ]:
_, _ = cv_auc_curve(
    X,
    y,
    lgboost_model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

In [ ]:
_, _ = cv_auc_curve(
    X,
    y,
    log_model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

## Check stratification

### Prep data

In [ ]:
random.seed(0)
# load the appointments data
appointments_df = load_appointment_csv("../data/raw/poliafspraken_pilot.csv")
appointments_df = process_appointments(appointments_df).sort_index()
with open("../output/models/no_show_model_cv.pickle", "rb") as f:
    model = pickle.load(f)

# create prediction scores and only select year 2022
all_postalcodes = process_postal_codes("../data/raw/NL.txt")
predictions_df = create_prediction(model, appointments_df, all_postalcodes)
predictions_df = predictions_df.loc[
    predictions_df.index.get_level_values("start").year == 2022
]
predictions_df = predictions_df.reset_index()

In [ ]:
# Create dummy hoofdagendas with the requirement (97% of same hoofdagenda for each pseudo_id)
groups = ["A", "B", "C"]
required_percentage = 0.97

def assign_group(df, groups, required_percentage):
    # Calculate the number of rows needed to satisfy the required percentage
    count = int(np.ceil(required_percentage * len(df)))
    # Choose a main group for the majority of entries
    main_group = np.random.choice(groups)
    # Assign the main group to the required percentage of rows
    df['hoofdagenda'] = main_group

    # Optionally, assign other groups to remaining rows
    remaining_indices = df.index[count:]  # indices for remaining rows
    if len(remaining_indices) > 0:
        df.loc[remaining_indices, 'hoofdagenda'] = np.random.choice(
            [g for g in groups if g != main_group], len(remaining_indices))

    return df

# Apply the function to each group of pseudo_id
predictions_df = predictions_df.groupby('pseudo_id').apply(assign_group, groups=groups,
                                                            required_percentage=required_percentage).reset_index(drop=True)

In [ ]:
print(predictions_df.shape)
predictions_df.head()

### create modified version of create_treatment group to test stratification

In [ ]:
pd.set_option('future.no_silent_downcasting', True)

# Function to apply the appropriate bin edges to each group
def apply_bins(group, bin_dict):
    edges = bin_dict[group.name]
    # Use pd.cut to segment the prediction values into bins based on the edges
    # 'labels=False' will return the indices of the bins from 0 to n_bins-1
    group['score_bin'] = pd.cut(group['prediction'],
                                 bins=[0] + list(edges.values())[1:] + [1],
                                   labels=False)
    return group

# create modified version of create_treatment_groups
def create_treatment_groups(
    predictions: pd.DataFrame, patients: pd.DataFrame, bin_edges
) -> pd.DataFrame:
    """
    Create treatment groups based on predictions.

    Parameters
    ----------
    predictions : pd.DataFrame
        DataFrame containing predictions.
    patients : pd.DataFrame
        A dataframe containing the treatment groups of patients
    bin_edges : list
        List of edges defining the bins for prediction scores.

    Returns
    -------
    pd.DataFrame
        DataFrame with treatment group assignments.

    Raises
    ------
    ValueError
        If the predictions DataFrame is empty.
    """
    if predictions.empty:
        raise ValueError("The predictions DataFrame is empty.")

    # get unique patient ids
    unique_patient_ids = predictions["pseudo_id"].unique().tolist()
    relevant_patients = patients[patients["id"].isin(unique_patient_ids)]

    if not relevant_patients.empty:
        # Merge predictions with patients to get treatment group
        predictions = pd.merge(
            predictions,
            relevant_patients[["id", "treatment_group"]],
            right_on="id",
            left_on="pseudo_id",
            how="left",
        )
        predictions.drop(columns=["id"], inplace=True)
    else:
        predictions.loc[:, "treatment_group"] = None

    predictions = predictions.groupby('hoofdagenda').apply(apply_bins,
                                                            bin_dict=bin_edges,
                                                            include_groups=False).reset_index()
    predictions = predictions.drop(columns="level_1")
    predictions = predictions.sort_values(["prediction"], ascending=False)

    # Fill NaN values in 'treatment_group' with calculated values
    mask = predictions["treatment_group"].isnull()
    predictions.loc[mask, 'treatment_group'] = (
        predictions[mask]
        .groupby(['hoofdagenda', 'score_bin'])['prediction']
        .transform(lambda x: (np.arange(len(x)) + random.randint(0, 1)) % 2)
    )

    # Apply mode calculation
    predictions.loc[mask, 'treatment_group'] = (
        predictions[mask]
        .groupby('pseudo_id')['treatment_group']
        .transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
    )

    # if new patients exist add them to the already existing patient table
    if len(predictions[mask]) > 0:
        new_patients = predictions[mask].drop_duplicates(subset='pseudo_id').copy()
        new_patients = new_patients.rename(columns={'pseudo_id': 'id'}).reset_index(drop=True)
        patients = pd.merge(patients, new_patients[['id', 'treatment_group']], on='id', how='outer', suffixes=('_df1', '_df2'))
        patients['treatment_group'] = patients['treatment_group_df1'].fillna(patients['treatment_group_df2'])
        patients.drop(columns=['treatment_group_df1', 'treatment_group_df2'], inplace=True)
    return predictions, patients

### Create prediction score bins for every first appointment of every patient and plot distribution

In [ ]:
# Calculate quantiles
n_bins = 4
quantiles = np.linspace(0, 1, n_bins + 1)
# determine quantiles for every hoofdagenda group in preditcions_df
bin_edges = (predictions_df.sort_values("prediction", ascending=False)
             .drop_duplicates(subset="pseudo_id", keep="first")
             .groupby('hoofdagenda')['prediction'].quantile(quantiles).reset_index())

bin_edges = pd.pivot_table(bin_edges, values='prediction', index='hoofdagenda', columns='level_1')
# create a dict where hoodagendas are keys and bin_edges for the quantiles are values
bin_edges = bin_edges.to_dict(orient='index')
print(bin_edges)

# Apply the function across the dataframe, grouped by 'hoofdagenda'
check = predictions_df.groupby('hoofdagenda').apply(apply_bins, bin_dict=bin_edges)
check = check.reset_index(drop=True)
# # plot distribution of each bin
fig, ax = plt.subplots()
check.groupby(["hoofdagenda", "score_bin"]).size().unstack().plot(kind='bar', ax=ax)
plt.xlabel("hoofdagenda and score_bin")
plt.ylabel("Count")
plt.title("Overal distribution of Treatment Groups per hoofdagenda and score_bin")
plt.show()

fig, ax = plt.subplots()
check.sort_values("prediction", ascending=False).drop_duplicates(subset="pseudo_id", keep="first").groupby(["hoofdagenda", "score_bin"]).size().unstack().plot(kind='bar', ax=ax)
plt.xlabel("hoofdagenda and score_bin")
plt.ylabel("Count")
plt.title("Distribution of Treatment Groups per hoofdagenda and score_bin for first appointment")
plt.show()

### Evaluate the stratification and visualize new distributions

In [ ]:
# iterate over every data
days = predictions_df["start"].dt.date.unique()
patients = pd.DataFrame(columns=["id", "treatment_group"])
final_predictions = []
for day in days:
    day_df = predictions_df[predictions_df["start"].dt.date == day]
    predict, patients = create_treatment_groups(day_df, patients, bin_edges)
    final_predictions.append(predict)

final_predictions= pd.concat(final_predictions)
final_predictions.shape

In [ ]:
# assert that every pseudo_id have a single unique treatment group
assert final_predictions.groupby('pseudo_id')['treatment_group'].nunique().eq(1).all()

# assert every pseudo_id has a treatment group
assert final_predictions['treatment_group'].notnull().all()


In [ ]:
# get first entry for every pseuoo_id
first_app = final_predictions.sort_values("prediction", ascending=False).drop_duplicates(subset="pseudo_id", keep="first")

In [ ]:
fig, ax = plt.subplots()
first_app.groupby(["hoofdagenda", "treatment_group", "score_bin"]).size().unstack().plot(kind='bar', ax=ax)
plt.xlabel("hoofdagenda and score_bin")
plt.ylabel("Count")
plt.title("Distribution of Treatment Groups per hoofdagenda and score_bin for first appointment")
plt.show()

In [ ]:
fig, ax = plt.subplots()
final_predictions.groupby(["hoofdagenda", "treatment_group", "score_bin"]).size().unstack().plot(kind='bar', ax=ax)
plt.xlabel("hoofdagenda and score_bin")
plt.ylabel("Count")
plt.title("Overall distribution of Treatment Groups per hoofdagenda and score_bin")
plt.show()


## Check temporal performance

In [ ]:
X_timesorted = X.sort_index(level="start")
y_timesorted = y.sort_index(level="start")

In [ ]:
roc_auc, test_indices = cv_auc_curve(
    X_timesorted,
    y_timesorted,
    lgboost_model,
    TimeSeriesSplit(n_splits=10),
)

In [ ]:
fold_times = [
    str(
        (
            X.iloc[idx].index.get_level_values("start").min().strftime("%Y-%m-%d"),
            X.iloc[idx].index.get_level_values("start").max().strftime("%Y-%m-%d"),
        )
    )
    for idx in test_indices.values()
]
fold_times

In [ ]:
roc_scores = pd.Series(roc_auc)
roc_scores.index = fold_times
roc_scores.plot.bar()

## Apply IECV

In [ ]:
appointments_df = load_appointment_csv("../data/raw/poliafspraken_no_show.csv")
appointments_df = process_appointments(appointments_df)
all_postalcodes = process_postal_codes("../data/raw/NL.txt")
appointments_features = create_features(
    appointments_df, all_postalcodes, minutes_early_cutoff=30
)

In [ ]:
appointments_features.columns

In [ ]:
appointments_features = (
    appointments_features[
        [
            "hoofdagenda",
            "hour",
            "weekday",
            "minutesDuration",
            "no_show",
            "prev_no_show",
            "prev_no_show_perc",
            "age",
            "dist_umcu",
            "prev_minutes_early",
            "earlier_appointments",
            "appointments_same_day",
            "appointments_last_days",
            "days_since_created",
            "days_since_last_appointment",
        ]
    ]
    .reset_index()
    .set_index(["pseudo_id", "start", "hoofdagenda"])
)

In [ ]:
appointments_features["no_show"] = (
    appointments_features["no_show"].replace({"no_show": 1, "show": 0}).astype(int)
)

appointments_features["hour"] = appointments_features["hour"].astype("category")
appointments_features["weekday"] = appointments_features["weekday"].astype("category")

X, y = appointments_features.drop(columns="no_show"), appointments_features["no_show"]

In [ ]:
train_groups = X.index.get_level_values("pseudo_id")

cv_auc_curve(
    X,
    y,
    lgboost_model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

In [ ]:
def group_leave_one_out(df):
    groups = df.index.get_level_values("hoofdagenda").unique()

    for test_group in groups:
        train_index = df.index.get_level_values("hoofdagenda") != test_group
        test_index = df.index.get_level_values("hoofdagenda") == test_group

        yield test_group, (np.where(train_index)[0], np.where(test_index)[0])

In [ ]:
def iecv_auc_curve(X_train, y_train, model):
    fpr = {}
    tpr = {}
    roc_auc = {}
    test_indices = {}

    fig, ax = plt.subplots()

    for group, (train_idx, test_idx) in group_leave_one_out(X_train):
        X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

        model.fit(X_train_cv, y_train_cv)

        y_score = model.predict_proba(X_test_cv)[:, 1]
        fpr[group], tpr[group], _ = roc_curve(y_test_cv, y_score)
        roc_auc[group] = roc_auc_score(y_test_cv, y_score)
        test_indices[group] = test_idx

        ax.plot(fpr[group], tpr[group], label=group)

    print(
        f"Mean AUC: {np.mean(list(roc_auc.values()))}"
        f"(+/- {np.std(list(roc_auc.values()))})"
    )
    ax.plot([0, 1], [0, 1], "k--")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.legend()
    fig.show()

    return roc_auc, test_indices

In [ ]:
iecv_auc_curve(X, y, lgboost_model)

In [ ]:
iecv_auc_curve(X, y, log_model)

## CV per poli

In [ ]:
X, y = appointments_features.drop(columns="no_show"), appointments_features["no_show"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0, shuffle=False
)
train_groups = X.index.get_level_values("pseudo_id")

In [ ]:
for poli in X.index.get_level_values("hoofdagenda").unique():
    X_tmp = X.loc[(slice(None), slice(None), poli), :]
    y_tmp = y.loc[(slice(None), slice(None), poli)]
    train_groups_tmp = X_tmp.index.get_level_values("pseudo_id")

    _, _ = cv_auc_curve(
        X_tmp,
        y_tmp,
        lgboost_model,
        # HistGradientBoostingClassifier(learning_rate=0.05, max_iter=300),
        StratifiedGroupKFold(n_splits=5),
        train_groups_tmp,
        title=poli,
    )

## Adding poli as feature

In [ ]:
appointments_features_agenda = appointments_features.reset_index()
appointments_features_agenda["hoofdagenda_cat"] = appointments_features_agenda[
    "hoofdagenda"
].astype("category")
appointments_features_agenda = appointments_features_agenda.set_index(
    ["pseudo_id", "start", "hoofdagenda"]
)
X, y = (
    appointments_features_agenda.drop(columns="no_show"),
    appointments_features_agenda["no_show"],
)

model = HistGradientBoostingClassifier(
    learning_rate=0.05,
    max_iter=300,
    categorical_features=["hour", "weekday", "hoofdagenda_cat"],
)
train_groups = X.index.get_level_values("pseudo_id")

In [ ]:
X.dtypes

In [ ]:
_, _ = cv_auc_curve(
    X,
    y,
    model,
    StratifiedGroupKFold(n_splits=5),
    train_groups,
)

## Relplots

In [ ]:
X, y = featuretable.drop(columns="no_show"), featuretable["no_show"]

model = log_model.fit(X, y)

In [ ]:
len(log_model[-1].coef_[0])

In [ ]:
import relplot as rp

y_pred_total = model.predict_proba(X)
print("calibration error:", rp.smECE(y_pred_total[:, 1], y))
fig, ax = rp.rel_diagram(y_pred_total[:, 1], y)
fig.show()

## Calculate permutation importance per feature

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(
    model,
    X,
    y,
    n_repeats=10,
    random_state=42,
)

In [ ]:
plt.rcdefaults()

feature_importance_df = pd.DataFrame(
    tuple(zip(X.columns, result["importances_mean"], strict=True))
)
feature_importance_df.columns = ["feature", "importance"]
feature_importance_df = feature_importance_df.sort_values("importance", ascending=False)
feature_importance_df.plot.barh(x="feature", y="importance")

In [ ]:
model = lgboost_model.fit(X, y)

In [ ]:
result = permutation_importance(
    model,
    X,
    y,
    n_repeats=10,
    random_state=42,
)

In [ ]:
feature_importance_df = pd.DataFrame(
    tuple(zip(X.columns, result["importances_mean"], strict=True))
)
feature_importance_df.columns = ["feature", "importance"]
feature_importance_df = feature_importance_df.sort_values("importance", ascending=False)
feature_importance_df.plot.barh(x="feature", y="importance")